In [69]:


!pip install -q /kaggle/input/rsna-bcd-whl-ds/python_gdcm-3.0.20-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -q /kaggle/input/rsna-bcd-whl-ds/pylibjpeg-1.4.0-py3-none-any.whl
!pip install -q /kaggle/input/rsna-bcd-whl-ds/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
!cp -r /kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle /tmp/ && pip install -q /tmp/efficientnet_kaggle



cp: cannot stat '/kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle': No such file or directory


In [70]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import tqdm
import dicomsdl as dicom


In [ ]:
train = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
test = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
train.head()
train = train[[col for col in train.columns if col in ['site_id',
 'patient_id',
 'image_id',
 'laterality',
 'view',
 'age',
 'implant',
 'machine_id',
 'prediction_id','cancer']]]
train.loc[train['age'].isnull(),'age'] = train['age'].mean()

path = os.path.join('/kaggle/input/rsna-breast-cancer-detection/train_images')
list_dir = os.listdir(path)
k =[]
y={}
mlo_path_l = []
cc_path_l = []
mlo_path_r = []
cc_path_r = []
for i, sub in tqdm.tqdm(enumerate(list_dir)):
    for j, fname in enumerate(os.listdir(os.path.join(path,sub))):
        view = (train.loc[train['image_id']==int(((fname).split('.'))[0]),'view'].values)[0]
        laterality = (train.loc[train['image_id']==int(((fname).split('.'))[0]),'laterality'].values)[0]
        cancer = int(train.loc[train['image_id']==int(((fname).split('.'))[0]),'cancer'])
        k.append(os.path.join(os.path.join(path,sub),fname))
        y[int(((fname).split('.'))[0])] = cancer
        if view =='MLO':
            if laterality=='L':
                 mlo_path_l.append(os.path.join(os.path.join(path,sub),fname))
            else:
                mlo_path_r.append(os.path.join(os.path.join(path,sub),fname))
        elif view =='CC':
            if laterality=='L':
                 cc_path_l.append(os.path.join(os.path.join(path,sub),fname))
            else:
                cc_path_r.append(os.path.join(os.path.join(path,sub),fname))
mlo_path_dict_l={}
for i in range(len(list(mlo_path_l))):
    mlo_path_dict_l[i] = mlo_path_l[i]



11455it [01:07, 174.85it/s]

tf.config.run_functions_eagerly(True)
@tf.function
def gen(x):
    i = path_dict[x]
    image = (dicom.open(i).pixelData(storedvalue=True))
    max_val = tf.cast(image[-1,1000],dtype=tf.float32)
    #print(max_val)
    la = dict_path[i]
    
    #if  train[train['image_id']==int(i.split('/')[-1].split('.')[0])]['laterality'].values[0]=='L':
    if la == 'L':
        if max_val>=1000:
            return (max_val-tf.image.resize(tf.expand_dims(image[100:-100,:2500],axis=-1),[150,100],preserve_aspect_ratio=False))/255.0
        else:
            return (tf.image.resize(tf.expand_dims(image[100:-100,:2500],axis=-1),[150,100],preserve_aspect_ratio=False)/255.0)
    elif la == 'R':
        if max_val>=1000:
            return (max_val-tf.image.resize(tf.image.flip_left_right
(tf.expand_dims(image[100:-100,:2500],axis=-1)),[150,100],preserve_aspect_ratio=False))/255.0
        else:
            return (tf.image.resize(tf.image.flip_left_right
(tf.expand_dims(image[100:-100,:2500],axis=-1)),[150,100],preserve_aspect_ratio=False)/255.0)
      #  if max_val>=1000:
      #      return ((max_val-tf.image.resize_with_pad(tf.image.flip_left_right(tf.expand_dims(image,axis=-1)),500,500))/255.0)[:,140:-60,0],(y[int(i.split('/')[-1].split('.')[0])])
      #  else:
       #     return (tf.image.resize_with_pad(tf.image.flip_left_right(tf.expand_dims(image,axis=-1)),500,500)/255.0)[:,140:-60,0],(y[int(i.split('/')[-1].split('.')[0])])



# Create a TensorFlow dataset and map your function to it
dataset= tf.data.Dataset.from_tensor_slices(np.arange((len(list(path_list)))))
dataset = dataset.map(lambda x: tf.numpy_function(gen, inp=[x], Tout=(tf.float32))).batch(1)                

In [ ]:
def pfbeta_fast(labels, predictions, beta=1):

    pTP = tf.math.reduce_sum(labels * predictions)
    pFP = tf.math.reduce_sum((1-labels) * predictions)
    num_positives = tf.math.reduce_sum((labels))  #  = pTP+pFN

    pPrecision = pTP/(pTP+pFP)
    pRecall = pTP/num_positives

    beta_squared = beta**2

    if (pPrecision > 0 and pRecall > 0):
        pF1 = (1+beta_squared) * pPrecision * pRecall/(beta_squared*pPrecision + pRecall)
        return pF1
    else:
        return 0.0


In [ ]:


def mylayer(X,f,size,s,how='same',a=1):
    layer = tf.keras.layers.Conv2D(filters=f,kernel_size=size,strides=s,padding=how)(X)
    layer = tf.keras.layers.BatchNormalization()(layer)
    if a ==1:
        layer = tf.keras.layers.Activation('relu')(layer)
    else:
        layer +=0
    return layer
def add_layer(X_prev,X):
    added = tf.keras.layers.Add()([X_prev,X])
    return added
def conv_connect(X_prev,X,f,s,how='same'):
    layer = tf.keras.layers.Conv2D(filters=f,kernel_size=(1,1),strides=2,padding=how)(X_prev)
    layer = tf.keras.layers.ZeroPadding2D((7,7))(layer)
    layer = tf.keras.layers.BatchNormalization()(layer)
    layer = add_layer(layer,X)
    layer = tf.keras.layers.Activation('relu')(layer)
    return layer

def cnn():
    
    
    pre = tf.keras.applications.convnext.ConvNeXtSmall(
    model_name='convnext_small',
    include_top=False,
    include_preprocessing=True,
    weights=None,
    input_shape=(150,100,3),pooling='avg')
    pre.load_weights('/kaggle/input/naidu-cancer-comp/convnext_small_notop.h5')
    pre.trainable=True
    
    for layer in pre.layers[:-20]:
        layer.trainable=False
    inputs = tf.keras.Input(shape=(150,100,1))
    X = tf.keras.layers.Concatenate()([inputs for i in range(3)])
    X = pre(X)
    #X = tf.keras.layers.Dropout(0.20)(X)
    outputs = tf.keras.layers.Dense(units=1, activation='sigmoid')(X)
    model = tf.keras.Model(inputs=inputs,outputs=outputs)
    model.summary()
    model.compile(loss='BinaryCrossentropy',optimizer = tf.keras.optimizers.Adam(),metrics=['accuracy',pfbeta_fast])
    return model



In [ ]:
tf.keras.backend.clear_session()
model = cnn()
model.load_weights('/kaggle/input/naidu-cancer-comp/mlo_l_10e.h5')
#model.fit(mlo_dataset,verbose=True,epochs=7,callbacks=[tf.keras.callbacks.ModelCheckpoint('model.h5')])
 

In [ ]:
test_sub

In [ ]:
test_sub = test[['image_id','view','laterality','patient_id','prediction_id']]
test_sub['path'] =0
for i in range(len(test_sub)):
    test_sub.loc[i,'path'] = '/kaggle/input/rsna-breast-cancer-detection/test_images/{}/{}.dcm'.format(str(test_sub.iloc[i,3]),str(test_sub.iloc[i,0]))

In [ ]:
for i,path in enumerate(test_sub['path']):
    image_id = path.split('/')[-1]
    sub = test_sub[test_sub['path']==path]#['laterality'].values[0]
    patient_id = path.split('/')[-2]
    if sub['laterality'].values[0] =='L':
        image = (dicom.open(path).pixelData(storedvalue=True))
        max_val = tf.cast(image[-1,1000],dtype=tf.float32)
        if max_val >=1000:
            image = (max_val-tf.image.resize(tf.expand_dims(image[100:-100,:2500],axis=-1),[150,100],preserve_aspect_ratio=False))/255.0
            test_sub.loc[i,'cancer']=model.predict(tf.expand_dims(image,0))[0,0]
        else:
            image = (max_val-tf.image.resize(tf.expand_dims(image[100:-100,:2500],axis=-1),[150,100],preserve_aspect_ratio=False))/255.0
            test_sub.loc[i,'cancer']=model.predict(tf.expand_dims(image,0))[0,0]
            
    else:
        image = ((dicom.open(path).pixelData(storedvalue=True)))
        max_val = tf.cast(image[-1,1000],dtype=tf.float32)
        if max_val >=1000:
            image = (max_val-tf.image.resize(tf.image.flip_left_right(tf.expand_dims(image,axis=-1))[100:-100,:2500,:],[150,100],preserve_aspect_ratio=False))/255.0
            test_sub.loc[i,'cancer']=model.predict(tf.expand_dims(image,0))[0,0]
        else:
            image = (tf.image.resize(tf.image.flip_left_right(tf.expand_dims(image,axis=-1))[100:-100,:2500,:],[150,100],preserve_aspect_ratio=False))/255.0
            test_sub.loc[i,'cancer']=model.predict(tf.expand_dims(image,0))[0,0]
        
        
        


In [ ]:
submission = ((test_sub.groupby(['prediction_id'])[['prediction_id','cancer']].sum()>0.5).astype('int').reset_index())
submission.to_csv('submission.csv',index=False)
submission
